In [9]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "AAC"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Ray Cookes Dashboard'))),
    html.Hr(),
    html.Div(
        
        
    dcc.Dropdown (
        id='breed types', options = [
            {'label': breed, 'value':breed} for breed in df['breed'].unique()
        ],
        multi = True,
        value = df['breed'].unique()
    ),
        
    dcc.RadioItems(
        id = 'filter type',
        options = [
            {'label': outcome, 'value': 'Adoption'},
            {'label': outcome, 'value': 'Transfer'},
            {'label': outcome, 'value': 'Return to Owner'},
            {'label': outcome, 'value': 'Euthanasia'},
            {'label': outcome, 'value': 'Rto-Adopt'},
            {'label': outcome, 'value': 'Died'},
            {'label': 'Water Rescue', 'value': 'water'}, 
            {'label': 'Disaster Tracking', 'value': 'helper},  
            {'label': 'Mountain/Wilderness Rescue', 'value': 'ranger'}, 
            {'label': 'Reset', 'value': 'Reset'},  
        ]

    )

        
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 

                        ),
    html.Hr(),
        dash_tale.DataTable(
        id = 'datatable-id',
        columns=[
            {'name': i, 'id': i, 'deletable': False, 'Selectable': True} for i in df.columns
        ],
            data.df.to_dict('records')
        )
    html.Br(),

#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    if filter_type == 'Adoption':
        df = pd.DataFrame(list(shelter.readAll({
            "outcome_type":{"$nin":["Adoption"]}
        })))
        
    if filter_type == 'Transfer':
        df = pd.DataFrame(list(shelter.readAll({
            "outcome_type":{"$nin":["Transfer"]}
        })))
        
    if filter_type == 'Return to Owner':
        df = pd.DataFrame(list(shelter.readAll({
            "outcome_type":{"$nin":["Return to Owner"]}
        })))
        
    if filter_type == 'Rto-Adopt':
        df = pd.DataFrame(list(shelter.readAll({
            "outcome_type":{"$nin":["Rto-Adopt"]}
        })))
        
    if filter_type == 'Died':
        df = pd.DataFrame(list(shelter.readAll({
            "outcome_type":{"$nin":["Died"]}
        })))
        
    if filter_type == 'Euthanasia':
        df = pd.DataFrame(list(shelter.readAll({
            "outcome_type":{"$nin":["Euthanasia"]}
        })))
             
    if filter_type == 'Water':
        df = pd.DataFrame(list(shelter.readAll({
            "animal_type": "Dog",
            "breed":{"$in":["Corgi", "Chow Chow"]},
            "outcome_type":{"$nin":["Adoption"]}
        })))
             
    if filter_type == 'helper':
        df = pd.DataFrame(list(shelter.readAll({
            "animal_type": "Dog",
            "breed":{"$in":["Chihuahua", "Boxer"]},
            "outcome_type":{"$nin":["Adoption"]}
        })))
             
    if filter_type == 'ranger':
        df = pd.DataFrame(list(shelter.readAll({
            "animal_type": "Dog",
            "breed":{"$in":["Pit Bull", "Westie"]},
            "outcome_type":{"$nin":["Adoption"]}
        })))
             
    if filter_type == 'reset':
        df = pd.DataFrame(list(shelter.readAll{}))
    
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
       
       
        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):

    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


SyntaxError: EOL while scanning string literal (3406605251.py, line 86)